# LangGraph Agent: LLM-Guided ReAct Workflow Executor

This notebook demonstrates a multi-agent LangGraph system:

- 🧠 `planner`: Generates a JSON playbook based on SOP
- 🤖 `decider`: Uses LLM to determine next tool(s) to execute
- 🛠️ `executor`: Calls tools from a registry

The system loops through the playbook nodes based on conditions and tool results.

In [ ]:
# Install LangChain and LangGraph if needed
# %pip install langchain langgraph openai
import json
from typing import TypedDict, List, Optional, Any, Dict, Callable
from langgraph.graph import StateGraph
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable
from langchain.chat_models import ChatOpenAI

# Mock LLM (replace with actual API)
llm = ChatOpenAI(model='gpt-4', temperature=0.1)

In [ ]:
# AgentState Definition
class AgentState(TypedDict):
    error_code: int
    sop_content: str
    input: str
    plan: dict
    executed_steps: List[str]
    current_step_index: Any
    tool_results: Optional[dict]
    data: Optional[Any]
    agent_outcome: Optional[str]

In [ ]:
from langchain.tools import tool

@tool
def timestamp_comparison_935():
    # Dummy logic
    return {"flag_1_count": 10, "flag_0_count": 5}

@tool
def create_rcl_file_935():
    return {"file": "RCL.xlsx"}

@tool
def create_research_file_935():
    return {"file": "research.xlsx"}

@tool
def send_email(file_type: str):
    return {"status": f"{file_type} file emailed"}
@tool
def create_servicenow_ticket(file_type: str):
    return {"status": f"{file_type} ticket created"}
@tool
def create_servicenow_ticket_935(file_type: str):
    return create_servicenow_ticket(file_type)
tool_registry = {
    "timestamp_comparison_935": timestamp_comparison_935,
    "create_rcl_file_935": create_rcl_file_935,
    "create_research_file_935": create_research_file_935,
    "send_email": send_email,
    "create_servicenow_ticket": create_servicenow_ticket
}


In [ ]:
def get_playbook_planner_node(llm, tool_descriptions) -> Callable[[AgentState], AgentState]:
    prompt = PromptTemplate.from_template("""
You are a planning agent. Given the SOP content and available tools, generate a workflow in JSON format.

SOP:
{sop}

Available Tools:
{tools}

Return a JSON object in this format:
{{
  "startNode": "node-1",
  "nodes": {{
    "node-1": {{
      "toolName": "...",
      "description": "...",
      "on_success": "..."
    }},
    ...
  }}
}}
""")

    chain = prompt | llm

    def plan(state: AgentState) -> AgentState:
        input_data = {
            "sop": state["sop_content"],
            "tools": tool_descriptions
        }
        response = chain.invoke(input_data)

        # 🔧 Extract text from AIMessage
        response_text = response.content if hasattr(response, "content") else str(response)

        # ✅ Parse JSON
        playbook = json.loads(response_text)

        state["plan"] = playbook
        state["current_step_index"] = playbook["startNode"]
        return state

    return plan


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable

def get_llm_path_decider_node(llm) -> Runnable:
    prompt = PromptTemplate.from_template("""
You are a flow controller for an agentic system.

Here is the JSON playbook plan (simplified):
{plan}

The last tool node executed was `{current_node_id}`.
Here is the result:
{tool_result}

Using this, decide the **next node or nodes to run**.

Respond in JSON as:
{{ "next_nodes": ["node-2"] }}
OR
{{ "next_nodes": ["node-2", "node-3"] }}  (if both branches should run)

If execution should stop, respond:
{{ "next_nodes": [] }}
""")

    chain = prompt | llm | (lambda output: json.loads(output))
    
    def decide_path(state: AgentState) -> AgentState:
        current_node = state["current_step_index"]
        plan = state["plan"]
        tool_result = state.get("tool_results", {})
        response = chain.invoke({
            "plan": json.dumps(plan["nodes"], indent=2),
            "current_node_id": current_node,
            "tool_result": json.dumps(tool_result)
        })
        state["agent_outcome"] = "Next: " + ", ".join(response["next_nodes"])
        state["current_step_index"] = response["next_nodes"]
        return state

    return decide_path


In [ ]:
def get_llm_react_executor_node(tool_registry: Dict[str, Callable]) -> Callable[[AgentState], AgentState]:
    def executor(state: AgentState) -> AgentState:
        plan = state["plan"]
        nodes = plan["nodes"]
        current_nodes = state.get("current_step_index", [plan["startNode"]])
        if isinstance(current_nodes, str):
            current_nodes = [current_nodes]
        executed = state.get("executed_steps", [])
        
        for node_id in current_nodes:
            if node_id in executed or node_id == "end":
                continue

            node = nodes.get(node_id)
            if not node:
                raise RuntimeError(f"Node '{node_id}' not found.")

            tool_name = node.get("toolName")
            if not tool_name:
                raise RuntimeError(f"No toolName in node '{node_id}'.")

            tool = tool_registry.get(tool_name)
            if not tool:
                raise RuntimeError(f"Tool '{tool_name}' not found in registry.")

            params = node.get("parameters", {})
            result = tool(**params)

            state["tool_results"] = result
            executed.append(node_id)

        state["executed_steps"] = executed
        return state

    return executor


In [ ]:
from langgraph.graph import StateGraph

builder = StateGraph(AgentState)

# Add all nodes
builder.add_node("planner", get_custom_json_planner_node(llm, available_tools))
builder.add_node("decider", get_llm_path_decider_node(llm))  # Reads the plan and tool results
builder.add_node("executor", get_llm_react_executor_node(tool_registry))  # Executes selected node(s)

# Correct flow:
builder.set_entry_point("planner")
builder.add_edge("planner", "decider")     # planner -> decider (read plan, get next nodes)
builder.add_edge("decider", "executor")    # decider -> executor (runs tools)
builder.add_edge("executor", "decider")    # loop executor -> decider
builder.set_finish_point("END")

builder.add_edge("executor", "decider")
builder.set_finish_point("decider")


In [ ]:
from langgraph.graph import END

def get_llm_path_decider_node(llm) -> Callable[[AgentState], Any]:
    prompt = PromptTemplate.from_template("""
You are a controller deciding the next step in a playbook.

Playbook:
{plan}
Current node: {current_node_id}
Tool result:
{tool_result}

Return {"next_nodes": ["node-2"]} for next steps, or [] to finish the plan.
""")

    chain = prompt | llm | (lambda x: json.loads(x))

    def decide(state: AgentState) -> Any:
        current = state["current_step_index"]
        if isinstance(current, list): current = current[0]

        response = chain.invoke({
            "plan": json.dumps(state["plan"], indent=2),
            "current_node_id": current,
            "tool_result": json.dumps(state.get("tool_results", {}))
        })

        next_nodes = response.get("next_nodes", [])

        if not next_nodes:
            return END  # 👈 This ends the graph

        state["current_step_index"] = next_nodes
        return state

    return decide
